In [93]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics.pairwise import cosine_similarity
import time
import gc  
import pprint
from rich import print
import plotly.express as px
from langdetect import detect, DetectorFactory
from sklearn.neighbors import KNeighborsClassifier
from tabulate import tabulate
import joblib
import warnings
warnings.filterwarnings("ignore")


## ===== PHASE 1: DATA LOADING & PREPROCESSING =====


In [94]:
start_time = time.time()

### Loading Data

In [95]:
df = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/data/raw/good_read_books_100k.csv", encoding='utf-8')


In [96]:
df.head(5)

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [97]:
df.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings'],
      dtype='object')

In [98]:
df.describe()

,pages,rating,reviews,totalratings
count,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,255.010240,3.833055,181.528450,2.990764e+03
std,367.913582,0.621237,1449.451229,3.635338e+04
min,0.000000,0.000000,0.000000,0.000000e+00
25%,135.000000,3.660000,3.000000,3.100000e+01
50%,240.000000,3.910000,15.000000,1.460000e+02
75%,336.000000,4.140000,67.000000,7.440000e+02
max,70000.000000,5.000000,158776.000000,3.819326e+06


In [99]:
# Read only necessary columns to save memory
cols_to_use = ['author', 'bookformat', 'genre', 'pages', 'rating', 'reviews', 'title', 'totalratings']
print(f"Original shape: {df.shape}")

Original shape: (100000, 13)

In [100]:
df = df.dropna(subset=['rating', 'genre', 'author'])
print(f"Shape after dropping NAs: {df.shape}")

Shape after dropping NAs: (89533, 13)

In [101]:
df.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings'],
      dtype='object')

In [102]:
df.describe()

,pages,rating,reviews,totalratings
count,89533.000000,89533.000000,89533.000000,8.953300e+04
mean,259.271174,3.890213,202.626897,3.339266e+03
std,338.807842,0.385779,1530.447003,3.840455e+04
min,0.000000,0.000000,0.000000,0.000000e+00
25%,144.000000,3.680000,5.000000,5.200000e+01
50%,242.000000,3.910000,20.000000,2.000000e+02
75%,338.000000,4.130000,81.000000,9.180000e+02
max,70000.000000,5.000000,158776.000000,3.819326e+06


In [103]:
# Initialize tracking dictionaries
model_metrics = {}
model_training_time = {}

In [104]:
# Reduce memory usage
df['pages'] = df['pages'].fillna(0).astype('int32')
df['reviews'] = df['reviews'].fillna(0).astype('int32')
df['totalratings'] = df['totalratings'].fillna(0).astype('int32')
df['rating'] = df['rating'].astype('float32')

In [105]:
df.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
5,Jeffrey Pfeffer,Hardcover,Why is common sense so uncommon when it comes ...,"Business,Leadership,Romance,Historical Romance...",https://i.gr-assets.com/images/S/compressed.ph...,875848419,9.78E+12,https://goodreads.com/book/show/1001090.The_Hu...,368,3.73,7,The Human Equation: Building Profits by Puttin...,119


In [106]:
# Create more informative features
df['log_pages'] = np.log1p(df['pages']).astype('float32')
df['log_reviews'] = np.log1p(df['reviews']).astype('float32')
df['log_totalratings'] = np.log1p(df['totalratings']).astype('float32')
df['popularity_score'] = df['rating'] * np.log1p(df['totalratings']).astype('float32')
df['review_ratio'] = (df['reviews'] / (df['totalratings'] + 1)).astype('float32')

In [107]:
len(df.columns)

18

In [108]:
# # One-hot encode genre (limited to top genres to save memory)
# genre_counts = df['genre'].value_counts().head(20)

In [109]:
# top_genres = genre_counts.index.tolist()
# df['top_genre'] = df['genre'].apply(lambda x: x if x in top_genres else 'Other')
# genre_ohe = pd.get_dummies(df['top_genre'], prefix='genre', sparse=True)
# df = pd.concat([df, genre_ohe], axis=1)

In [110]:
# Target variable
target = (df['rating'] >= 4.0).astype('int8')
print(f"Target distribution: {np.bincount(target)}")

Target distribution: [52913 36620]

In [111]:
df.isna().sum()

author                  0
bookformat           2324
desc                 4048
genre                   0
img                  1224
isbn                12624
isbn13               9789
link                    0
pages                   0
rating                  0
reviews                 0
title                   1
totalratings            0
log_pages               0
log_reviews             0
log_totalratings        0
popularity_score        0
review_ratio            0
dtype: int64

In [112]:
df['title']

0        Between Two Fires: American Indians in the Civ...
1                                 Fashion Sourcebook 1920s
2                                               Hungary 56
3        All-American Anarchist: Joseph A. Labadie and ...
5        The Human Equation: Building Profits by Puttin...
                               ...                        
99993                                       The Sea Inside
99994                                    A Horse for Angel
99997    A Faith Worth Sharing: A Lifetime of Conversat...
99998    A Volcano Beneath the Snow: John Brown's War A...
99999    Paranormal Nation: Why America Needs Ghosts, U...
Name: title, Length: 89533, dtype: object

In [113]:
# Sample: apply language detection
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'


In [114]:
# Detect languages
df['language'] = df['title'].apply(detect_language)
df['language'].value_counts()


language
en         67549
de          3694
nl          1429
da          1428
pt          1363
fr          1344
it          1140
id          1118
ro           984
tl           940
af           923
no           832
es           720
ca           681
tr           517
et           508
fi           501
sv           479
cy           444
pl           370
sw           365
so           326
vi           278
hu           260
sl           235
lt           230
hr           212
unknown      144
sk           138
sq           130
cs           124
zh-cn         68
lv            42
ko            17
Name: count, dtype: int64

In [115]:
# Set a threshold for suspicious languages (e.g., languages with < 100 titles)
suspicious_langs = df['language'].value_counts()
suspicious_langs = suspicious_langs[suspicious_langs < 100].index.tolist()

# Filter rows with suspicious language codes
suspicious_titles = df[df['language'].isin(suspicious_langs)]

# View them (e.g., top 30)
print(suspicious_titles[['title', 'language']].head(5))


title language
344            Ð¢Ð¾Ñ€ÐµÐ°Ð´Ð¾Ñ€Ð¸ Ð· Ð’Ð°ÑÑŽÐºÑ–Ð²ÐºÐ¸       ko
4814  ÐÐ¾Ñ‡Ð½Ð¾Ð¹ Ð´Ð¾Ð·Ð¾Ñ€. Ð”Ð½ÐµÐ²Ð½Ð¾Ð¹ Ð´Ð¾Ð·...    zh-cn
4881                                      Dinosaurumpus       lv
6393  100 ÐºÐ°Ð·Ð¾Ðº. ÐÐ°Ð¹ÐºÑ€Ð°Ñ‰Ñ– ÑƒÐºÑ€Ð°Ñ—Ð½Ñ...    zh-cn
7169         Ð›ÐµÐºÑÐ¸ÐºÐ¾Ð½ Ñ‚Ð°Ñ”Ð¼Ð½Ð¸Ñ… Ð·Ð½Ð°Ð½ÑŒ    zh-cn

In [116]:

# Step 2: Drop rows where the language is in the suspicious list
df = df[~df['language'].isin(suspicious_langs)].reset_index(drop=True)

# ===== PHASE 3: ENHANCED FEATURE ENGINEERING =====

#### Columns before One-hot encoding

In [117]:
df.columns

Index(['author', 'bookformat', 'desc', 'genre', 'img', 'isbn', 'isbn13',
       'link', 'pages', 'rating', 'reviews', 'title', 'totalratings',
       'log_pages', 'log_reviews', 'log_totalratings', 'popularity_score',
       'review_ratio', 'language'],
      dtype='object')

In [118]:
# ===== PHASE 2: ENHANCED FEATURE ENGINEERING =====
print("Performing feature engineering...")
# Encode categorical features
genre_encoder = LabelEncoder()
author_encoder = LabelEncoder()
format_encoder = LabelEncoder()

df['genre_encoded'] = genre_encoder.fit_transform(df['genre'])
df['author_encoded'] = author_encoder.fit_transform(df['author'])
df['bookformat'] = df['bookformat'].fillna('Unknown')
df['format_encoded'] = format_encoder.fit_transform(df['bookformat'])

# Create more informative features
df['log_pages'] = np.log1p(df['pages']).astype('float32')
df['log_reviews'] = np.log1p(df['reviews']).astype('float32')
df['log_totalratings'] = np.log1p(df['totalratings']).astype('float32')
df['popularity_score'] = df['rating'] * np.log1p(df['totalratings']).astype('float32')
df['review_ratio'] = (df['reviews'] / (df['totalratings'] + 1)).astype('float32')

# One-hot encode genre (limited to top genres to save memory)
genre_counts = df['genre'].value_counts().head(20)
top_genres = genre_counts.index.tolist()
df['top_genre'] = df['genre'].apply(lambda x: x if x in top_genres else 'Other')
genre_ohe = pd.get_dummies(df['top_genre'], prefix='genre', sparse=True)
df = pd.concat([df, genre_ohe], axis=1)

# Target variable
target = (df['rating'] >= 4.0).astype('int8')
print(f"Target distribution: {np.bincount(target)}")


Performing feature engineering...

Target distribution: [52857 36549]

#### Columns after One-hot encoding

In [119]:
len(df.columns)

44

In [120]:
# ===== PHASE 3: PREPARE TRAINING DATA =====
print("Preparing training data...")
# Select features with better predictive power
numerical_features = [
    'log_pages', 'log_reviews', 'log_totalratings', 
    'popularity_score', 'review_ratio'
]

categorical_features = ['author_encoded', 'format_encoded']

Preparing training data...

In [121]:
# Combine all features, including one-hot encoded genres
feature_cols = numerical_features + categorical_features + list(genre_ohe.columns)
features = df[feature_cols].copy()

In [122]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42, stratify=target
)

In [123]:
# Scale numerical features only
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

In [124]:
# Memory cleanup
del genre_ohe
gc.collect()

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (71524, 28)

Test data shape: (17882, 28)

In [125]:
# ===== MODEL 1: XGBOOST WITH EPOCHS =====
print("\n=== Training XGBoost Model ===")
start_time = time.time()

=== Training XGBoost Model ===

#### Function to print metrics (reusable)

In [126]:
def print_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [127]:
import xgboost as xgb
from tabulate import tabulate

def train_xgboost_with_epochs(X_train, y_train, X_test, y_test, epochs_list=[3, 10, 50, 100]):
    results = {}
    best_accuracy = 0
    best_model = None
    best_epoch = 0

    print("\nTraining and Evaluating XGBoost Models...\n")
    
    for epochs in epochs_list:
        print(f"Training XGBoost with {epochs} estimators...")
        model = xgb.XGBClassifier(
            n_estimators=epochs,
            learning_rate=0.1,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric='logloss',
            use_label_encoder=False,
            random_state=42
        )

        # Fit model (eval_set given, early stopping removed for this version)
        eval_set = [(X_test, y_test)]
        model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

        # Evaluate
        y_pred = model.predict(X_test)
        metrics = print_metrics(y_test, y_pred, f"XGBoost ({epochs} epochs)")
        results[epochs] = metrics

        # Track best
        if metrics["accuracy"] > best_accuracy:
            best_accuracy = metrics["accuracy"]
            best_model = model
            best_epoch = epochs

    # Tabulate the results
    table = []
    for epochs, metrics in results.items():
        row = [epochs] + [round(metrics.get(m, 0), 4) for m in ["accuracy", "precision", "recall", "f1_score"]]
        table.append(row)

    headers = ["Epochs", "Accuracy", "Precision", "Recall", "F1 Score"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

    print(f"\n✅ Best XGBoost model: {best_epoch} epochs with accuracy {best_accuracy:.4f}")
    return best_model, results


In [128]:
# Train XGBoost with different epochs
xgb_model, xgb_results = train_xgboost_with_epochs(
    X_train, y_train, X_test, y_test, 
    epochs_list=[3, 10, 50, 100, 200]
)

Training and Evaluating XGBoost Models...

Training XGBoost with 3 estimators...

XGBoost (3 epochs) - Accuracy: 0.5963, Precision: 0.9789, Recall: 0.0127, F1: 0.0251

Training XGBoost with 10 estimators...

XGBoost (10 epochs) - Accuracy: 0.7314, Precision: 0.9321, Recall: 0.3699, F1: 0.5296

...

XGBoost (100 epochs) - Accuracy: 0.9592, Precision: 0.9603, Recall: 0.9390, F1: 0.9495

Training XGBoost with 200 estimators...

XGBoost (200 epochs) - Accuracy: 0.9777, Precision: 0.9739, Recall: 0.9715, F1: 0.9727

+----------+------------+-------------+----------+------------+
|   Epochs |   Accuracy |   Precision |   Recall |   F1 Score |
+==========+============+=============+==========+============+
|        3 |     0.5963 |      0.9789 |   0.0127 |          0 |
+----------+------------+-------------+----------+------------+
|       10 |     0.7314 |      0.9321 |   0.3699 |          0 |
+----------+------------+-------------+----------+------------+
|       50 |     0.9082 |      0.9431 |   0.8252 |          0 |
+----------+------------+-------------+----------+------------+
|      100 |     0.9592 |      0.9603 |   0.939  |          0 |
+----------+------------+-------------+----------+------------+
|      200 |     0.9777 |      0.9739 |   0.9715 |          0 |
+----------+------------+-------------+----------+------------+

✅ Best XGBoost model: 200 epochs with accuracy 0.9777

In [129]:
# Store metrics
model_metrics["XGBoost"] = xgb_results
model_training_time["XGBoost"] = time.time() - start_time

In [130]:

# Memory cleanup
gc.collect()

35

In [131]:
# ===== MODEL 2: KMEANS CLUSTERING =====
print("\n=== Training KMeans Model ===")
start_time = time.time()

=== Training KMeans Model ===

In [132]:

def train_kmeans_with_different_clusters(X_train, y_train, X_test, y_test, clusters_list=[3, 5, 8, 10]):
    results = {}
    sample_size = min(10000, len(X_train))
    X_train_sample = X_train.iloc[:sample_size].copy()
    y_train_sample = y_train[:sample_size].copy()

    print("\nTraining and Evaluating KMeans Models...\n")
    for n_clusters in clusters_list:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        kmeans.fit(X_train_sample)
        
        # Assign cluster to each point
        train_clusters = kmeans.predict(X_train_sample)
        
        # For each cluster, determine the majority class
        cluster_to_label = {}
        for cluster in range(n_clusters):
            cluster_points = (train_clusters == cluster)
            if sum(cluster_points) > 0:
                labels = y_train_sample[cluster_points]
                majority_label = 1 if sum(labels) > len(labels)/2 else 0
                cluster_to_label[cluster] = majority_label
        
        # Predict using cluster assignments
        test_clusters = kmeans.predict(X_test)
        y_pred = np.array([cluster_to_label.get(cluster, 0) for cluster in test_clusters])
        
        # Evaluate
        metrics = print_metrics(y_test, y_pred, f"KMeans ({n_clusters} clusters)")
        results[n_clusters] = metrics

    # Tabulate the results
    table = []
    for n_clusters, metrics in results.items():
        row = [n_clusters] + [round(metrics.get(m, 0), 4) for m in ["accuracy", "precision", "recall", "f1_score"]]
        table.append(row)

    headers = ["Clusters", "Accuracy", "Precision", "Recall", "F1 Score"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

    # Find best number of clusters
    best_n_clusters = max(results.items(), key=lambda x: x[1]["accuracy"])[0]
    print(f"\nBest KMeans model: {best_n_clusters} clusters with accuracy {results[best_n_clusters]['accuracy']:.4f}")
    
    # Train final model with best clusters
    best_model = KMeans(n_clusters=best_n_clusters, random_state=42, n_init=10)
    best_model.fit(X_train_sample)
    return best_model, results

In [133]:
# Train KMeans with different numbers of clusters
kmeans_model, kmeans_results = train_kmeans_with_different_clusters(
    X_train, y_train, X_test, y_test,
    clusters_list=[3, 5, 8, 10, 15]
)

Training and Evaluating KMeans Models...

KMeans (3 clusters) - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

KMeans (5 clusters) - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

KMeans (8 clusters) - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

KMeans (10 clusters) - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

KMeans (15 clusters) - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

+------------+------------+-------------+----------+------------+
|   Clusters |   Accuracy |   Precision |   Recall |   F1 Score |
+============+============+=============+==========+============+
|          3 |     0.5912 |           0 |        0 |          0 |
+------------+------------+-------------+----------+------------+
|          5 |     0.5912 |           0 |        0 |          0 |
+------------+------------+-------------+----------+------------+
|          8 |     0.5912 |           0 |        0 |          0 |
+------------+------------+-------------+----------+------------+
|         10 |     0.5912 |           0 |        0 |          0 |
+------------+------------+-------------+----------+------------+
|         15 |     0.5912 |           0 |        0 |          0 |
+------------+------------+-------------+----------+------------+

Best KMeans model: 3 clusters with accuracy 0.5912

In [134]:
# Store metrics
model_metrics["KMeans"] = kmeans_results
model_training_time["KMeans"] = time.time() - start_time

In [135]:
# Memory cleanup
gc.collect()

0

In [136]:
# ===== MODEL 3: KNN WITH DIFFERENT K VALUES =====
print("\n=== Training KNN Model ===")
start_time = time.time()

=== Training KNN Model ===

In [137]:

# Function to train KNN with different K values and display results using tabulate
def train_knn_with_different_k(X_train, y_train, X_test, y_test, k_values=[1, 3, 5, 7, 10]):
    results = {}
    best_accuracy = 0
    best_model = None
    best_k = 0

    # Use a smaller sample for memory efficiency
    sample_size = min(10000, len(X_train))
    X_train_sample = X_train.iloc[:sample_size].copy()
    y_train_sample = y_train[:sample_size].copy()

    print("\nTraining and Evaluating KNN Models...\n")
    for k in k_values:
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train_sample, y_train_sample)

        # Evaluate on test data
        y_pred = model.predict(X_test)
        metrics = print_metrics(y_test, y_pred, f"KNN (k={k})")
        results[k] = metrics

        # Track best model
        if metrics["accuracy"] > best_accuracy:
            best_accuracy = metrics["accuracy"]
            best_model = model
            best_k = k

    # Tabulate the results
    table = []
    for k, metrics in results.items():
        row = [k] + [round(metrics.get(m, 0), 4) for m in ["accuracy", "precision", "recall", "f1_score"]]
        table.append(row)

    headers = ["k", "Accuracy", "Precision", "Recall", "F1 Score"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

    print(f"\nBest KNN model: k={best_k} with accuracy {best_accuracy:.4f}")
    return best_model, results


In [138]:
# Train KNN with different K values
knn_model, knn_results = train_knn_with_different_k(
    X_train, y_train, X_test, y_test,
    k_values=[1, 3, 5, 7, 11, 15]
)

Training and Evaluating KNN Models...

KNN (k=1) - Accuracy: 0.5609, Precision: 0.4635, Recall: 0.4703, F1: 0.4669

KNN (k=3) - Accuracy: 0.5514, Precision: 0.4474, Recall: 0.4146, F1: 0.4304

KNN (k=5) - Accuracy: 0.5597, Precision: 0.4530, Recall: 0.3720, F1: 0.4085

KNN (k=7) - Accuracy: 0.5622, Precision: 0.4540, Recall: 0.3506, F1: 0.3957

KNN (k=11) - Accuracy: 0.5619, Precision: 0.4481, Recall: 0.3092, F1: 0.3659

KNN (k=15) - Accuracy: 0.5653, Precision: 0.4496, Recall: 0.2826, F1: 0.3471

+-----+------------+-------------+----------+------------+
|   k |   Accuracy |   Precision |   Recall |   F1 Score |
+=====+============+=============+==========+============+
|   1 |     0.5609 |      0.4635 |   0.4703 |          0 |
+-----+------------+-------------+----------+------------+
|   3 |     0.5514 |      0.4474 |   0.4146 |          0 |
+-----+------------+-------------+----------+------------+
|   5 |     0.5597 |      0.453  |   0.372  |          0 |
+-----+------------+-------------+----------+------------+
|   7 |     0.5622 |      0.454  |   0.3506 |          0 |
+-----+------------+-------------+----------+------------+
|  11 |     0.5619 |      0.4481 |   0.3092 |          0 |
+-----+------------+-------------+----------+------------+
|  15 |     0.5653 |      0.4496 |   0.2826 |          0 |
+-----+------------+-------------+----------+------------+

Best KNN model: k=15 with accuracy 0.5653

In [139]:
# Store metrics
model_metrics["KNN"] = knn_results
model_training_time["KNN"] = time.time() - start_time

In [140]:
# Memory cleanup
gc.collect()

0

In [141]:
# ===== MODEL 4: SVD DIMENSIONALITY REDUCTION =====
print("\n=== Training SVD Model ===")
start_time = time.time()

=== Training SVD Model ===

In [142]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from tabulate import tabulate
import numpy as np

def train_svd_with_different_components(X_train, y_train, X_test, y_test, n_components_list=None):
    # Get the maximum number of components possible
    max_components = min(X_train.shape[0], X_train.shape[1]) - 1
    
    if n_components_list is None:
        if max_components <= 5:
            n_components_list = [2, max_components]
        elif max_components <= 10:
            n_components_list = [2, 5, max_components]
        elif max_components <= 20:
            n_components_list = [2, 5, 10, max_components]
        else:
            n_components_list = [2, 5, 10, max(15, max_components // 4), max(20, max_components // 2)]
    else:
        n_components_list = [n for n in n_components_list if n < max_components]

    print(f"Testing SVD with components: {n_components_list} (max possible: {max_components})")

    results = {}
    best_accuracy = 0
    best_model = None
    best_n_components = 0

    for n_components in n_components_list:
        print(f"\nTraining SVD with {n_components} components...")
        
        # Dimensionality reduction
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        X_train_svd = svd.fit_transform(X_train)
        X_test_svd = svd.transform(X_test)

        # Train classifier
        clf = LogisticRegression(max_iter=1000, random_state=42)
        clf.fit(X_train_svd, y_train)

        # Evaluate
        y_pred = clf.predict(X_test_svd)
        metrics = print_metrics(y_test, y_pred, f"SVD ({n_components} components) + LR")
        results[n_components] = metrics

        # Track best
        if metrics["accuracy"] > best_accuracy:
            best_accuracy = metrics["accuracy"]
            best_model = (svd, clf)
            best_n_components = n_components

    # Tabulate the results
    table = []
    for n_components, metrics in results.items():
        row = [n_components] + [round(metrics.get(m, 0), 4) for m in ["accuracy", "precision", "recall", "f1_score"]]
        table.append(row)

    headers = ["Components", "Accuracy", "Precision", "Recall", "F1 Score"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

    print(f"\nBest SVD model: {best_n_components} components with accuracy {best_accuracy:.4f}")
    return best_model, results


In [143]:
# Train SVD with different components
svd_models, svd_results = train_svd_with_different_components(
    X_train, y_train, X_test, y_test,
    n_components_list=[2, 5, 10, 15, 20, 25]  # Reduced max components to 25
)


Testing SVD with components: [2, 5, 10, 15, 20, 25] (max possible: 27)

Training SVD with 2 components...

SVD (2 components) + LR - Accuracy: 0.5912, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

Training SVD with 5 components...

SVD (5 components) + LR - Accuracy: 0.6256, Precision: 0.6371, Recall: 0.1955, F1: 0.2992

...

SVD (20 components) + LR - Accuracy: 0.9673, Precision: 0.9551, Recall: 0.9654, F1: 0.9602

Training SVD with 25 components...

SVD (25 components) + LR - Accuracy: 0.9810, Precision: 0.9770, Recall: 0.9765, F1: 0.9767

+--------------+------------+-------------+----------+------------+
|   Components |   Accuracy |   Precision |   Recall |   F1 Score |
+==============+============+=============+==========+============+
|            2 |     0.5912 |      0      |   0      |          0 |
+--------------+------------+-------------+----------+------------+
|            5 |     0.6256 |      0.6371 |   0.1955 |          0 |
+--------------+------------+-------------+----------+------------+
|           10 |     0.9795 |      0.9735 |   0.9765 |          0 |
+--------------+------------+-------------+----------+------------+
|           15 |     0.9729 |      0.9691 |   0.9644 |          0 |
+--------------+------------+-------------+----------+------------+
|           20 |     0.9673 |      0.9551 |   0.9654 |          0 |
+--------------+------------+-------------+----------+------------+
|           25 |     0.981  |      0.977  |   0.9765 |          0 |
+--------------+------------+-------------+----------+------------+

Best SVD model: 25 components with accuracy 0.9810

In [144]:
# Store metrics
model_metrics["SVD"] = svd_results
model_training_time["SVD"] = time.time() - start_time

In [145]:
# Memory cleanup
gc.collect()

0

In [146]:
# ===== MODEL 5: LOGISTIC REGRESSION WITH DIFFERENT REGULARIZATION =====
print("\n=== Training Logistic Regression Model ===")
start_time = time.time()

=== Training Logistic Regression Model ===

In [147]:
from sklearn.linear_model import LogisticRegression
from tabulate import tabulate

def train_logreg_with_different_params(X_train, y_train, X_test, y_test):
    results = {}
    best_accuracy = 0
    best_model = None
    best_params = None

    param_grid = [
        {'C': 0.01, 'solver': 'liblinear'},
        {'C': 0.1, 'solver': 'liblinear'},
        {'C': 1.0, 'solver': 'liblinear'},
        {'C': 10.0, 'solver': 'liblinear'},
        {'C': 100.0, 'solver': 'liblinear'}
    ]

    print("\nTraining and Evaluating Logistic Regression Models...\n")
    for params in param_grid:
        c_val = params['C']
        solver = params['solver']
        model = LogisticRegression(C=c_val, solver=solver, max_iter=1000, random_state=42)
        model.fit(X_train, y_train)

        # Evaluate
        y_pred = model.predict(X_test)
        metrics = print_metrics(y_test, y_pred, f"LogReg (C={c_val})")
        results[c_val] = metrics

        # Track best model
        if metrics["accuracy"] > best_accuracy:
            best_accuracy = metrics["accuracy"]
            best_model = model
            best_params = params

    # Tabulate the results
    table = []
    for c_val, metrics in results.items():
        row = [c_val] + [round(metrics.get(m, 0), 4) for m in ["accuracy", "precision", "recall", "f1_score"]]
        table.append(row)

    headers = ["C", "Accuracy", "Precision", "Recall", "F1 Score"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

    print(f"\nBest Logistic Regression model: C={best_params['C']} with accuracy {best_accuracy:.4f}")
    return best_model, results


In [148]:
# Train Logistic Regression with different parameters
logreg_model, logreg_results = train_logreg_with_different_params(
    X_train, y_train, X_test, y_test
)

Training and Evaluating Logistic Regression Models...

LogReg (C=0.01) - Accuracy: 0.6479, Precision: 0.6903, Recall: 0.2516, F1: 0.3688

LogReg (C=0.1) - Accuracy: 0.6479, Precision: 0.6901, Recall: 0.2516, F1: 0.3687

LogReg (C=1.0) - Accuracy: 0.6479, Precision: 0.6901, Recall: 0.2516, F1: 0.3687

LogReg (C=10.0) - Accuracy: 0.6479, Precision: 0.6901, Recall: 0.2516, F1: 0.3687

LogReg (C=100.0) - Accuracy: 0.6479, Precision: 0.6901, Recall: 0.2516, F1: 0.3687

+--------+------------+-------------+----------+------------+
|      C |   Accuracy |   Precision |   Recall |   F1 Score |
+========+============+=============+==========+============+
|   0.01 |     0.6479 |      0.6903 |   0.2516 |          0 |
+--------+------------+-------------+----------+------------+
|   0.1  |     0.6479 |      0.6901 |   0.2516 |          0 |
+--------+------------+-------------+----------+------------+
|   1    |     0.6479 |      0.6901 |   0.2516 |          0 |
+--------+------------+-------------+----------+------------+
|  10    |     0.6479 |      0.6901 |   0.2516 |          0 |
+--------+------------+-------------+----------+------------+
| 100    |     0.6479 |      0.6901 |   0.2516 |          0 |
+--------+------------+-------------+----------+------------+

Best Logistic Regression model: C=0.01 with accuracy 0.6479

In [149]:
# Store metrics
model_metrics["LogisticRegression"] = logreg_results
model_training_time["LogisticRegression"] = time.time() - start_time

In [150]:
# Memory cleanup
gc.collect()

0

In [151]:
# ===== MODEL COMPARISON AND VISUALIZATION =====
print("\n=== Model Comparison ===")

=== Model Comparison ===

In [152]:
def visualize_model_comparison():
    # Prepare data for visualization
    models = []
    accuracy_scores = []
    f1_scores = []
    training_times = []

    # Extract best scores for each model
    for model_name, results in model_metrics.items():
        if isinstance(results, dict):
            best_config = max(results.items(), key=lambda x: x[1]["accuracy"])
            best_param = best_config[0]
            best_metrics = best_config[1]

            full_name = f"{model_name} ({best_param})"
            models.append(full_name)
            accuracy_scores.append(best_metrics["accuracy"])
            f1_scores.append(best_metrics["f1"])
            training_times.append(model_training_time.get(model_name, 0))
        else:
            models.append(model_name)
            accuracy_scores.append(results["accuracy"])
            f1_scores.append(results["f1"])
            training_times.append(model_training_time.get(model_name, 0))

    # Build tabular data
    table_data = []
    for i in range(len(models)):
        table_data.append([
            models[i],
            round(accuracy_scores[i], 4),
            round(f1_scores[i], 4),
            round(training_times[i], 2)
        ])

    # Define headers
    headers = ["Model", "Accuracy", "F1 Score", "Training Time (s)"]

    # Print the comparison table
    print("\nModel Performance Comparison:")
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    # Best model by accuracy
    best_idx = np.argmax(accuracy_scores)
    print(f"\n✅ Best Model: {models[best_idx]} "
          f"with accuracy {accuracy_scores[best_idx]:.4f} "
          f"and F1 {f1_scores[best_idx]:.4f}")

    # Time-efficient model (>= 80% of best accuracy)
    threshold = 0.8 * max(accuracy_scores)
    valid_indices = [i for i, acc in enumerate(accuracy_scores) if acc >= threshold]
    if valid_indices:
        most_efficient_idx = min(valid_indices, key=lambda i: training_times[i])
        print(f"⚡ Most Time-Efficient Model: {models[most_efficient_idx]} "
              f"with accuracy {accuracy_scores[most_efficient_idx]:.4f} "
              f"and training time {training_times[most_efficient_idx]:.2f}s")


In [153]:
# Visualize model comparison
visualize_model_comparison()

Model Performance Comparison:

+---------------------------+------------+------------+---------------------+
| Model                     |   Accuracy |   F1 Score |   Training Time (s) |
+===========================+============+============+=====================+
| XGBoost (200)             |     0.9777 |     0.9727 |                1.26 |
+---------------------------+------------+------------+---------------------+
| KMeans (3)                |     0.5912 |     0      |                4.5  |
+---------------------------+------------+------------+---------------------+
| KNN (15)                  |     0.5653 |     0.3471 |                1.85 |
+---------------------------+------------+------------+---------------------+
| SVD (25)                  |     0.981  |     0.9767 |               17.69 |
+---------------------------+------------+------------+---------------------+
| LogisticRegression (0.01) |     0.6479 |     0.3688 |                0.67 |
+---------------------------+------------+------------+---------------------+

✅ Best Model: SVD (25) with accuracy 0.9810 and F1 0.9767

⚡ Most Time-Efficient Model: XGBoost (200) with accuracy 0.9777 and training time 1.26s

In [154]:
# ===== MODEL PARAMETER TUNING VISUALIZATION =====
# Function to visualize parameter tuning results
def visualize_parameter_tuning(model_name, param_results, param_name="Parameter"):
    if not param_results:
        print(f"No parameter tuning results available for {model_name}")
        return
    
    print(f"\n{model_name} Parameter Tuning Results:")
    print("="*50)
    print(f"{param_name:<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1 Score':<10}")
    print("-"*50)
    
    for param, metrics in sorted(param_results.items()):
        print(f"{param:<15} {metrics['accuracy']:<10.4f} {metrics['precision']:<10.4f} "
              f"{metrics['recall']:<10.4f} {metrics['f1']:<10.4f}")
    
    # Find best parameter
    best_param = max(param_results.items(), key=lambda x: x[1]["accuracy"])[0]
    print("="*50)
    print(f"Best {param_name}: {best_param} with accuracy {param_results[best_param]['accuracy']:.4f}")

# Visualize parameter tuning for each model
print("\n=== Parameter Tuning Results ===")
if "XGBoost" in model_metrics:
    visualize_parameter_tuning("XGBoost", model_metrics["XGBoost"], "Epochs")
if "KMeans" in model_metrics:
    visualize_parameter_tuning("KMeans", model_metrics["KMeans"], "Clusters")
if "KNN" in model_metrics:
    visualize_parameter_tuning("KNN", model_metrics["KNN"], "K Value")
if "SVD" in model_metrics:
    visualize_parameter_tuning("SVD", model_metrics["SVD"], "Components")
if "LogisticRegression" in model_metrics:
    visualize_parameter_tuning("LogisticRegression", model_metrics["LogisticRegression"], "C Value")

=== Parameter Tuning Results ===

XGBoost Parameter Tuning Results:

==================================================

Epochs          Accuracy   Precision  Recall     F1 Score

--------------------------------------------------

...

1.0             0.6479     0.6901     0.2516     0.3687

10.0            0.6479     0.6901     0.2516     0.3687

100.0           0.6479     0.6901     0.2516     0.3687

==================================================

Best C Value: 0.01 with accuracy 0.6479

In [155]:
# ===== RECOMMENDATION FUNCTIONS =====
print("\n=== Generating Recommendations ===")

def content_based_recommendation(book_id, df, feature_matrix, n=5):
    """Generate recommendations based on book similarity"""
    # Ensure book_id is valid
    if book_id >= len(feature_matrix):
        book_id = 0
        
    # Get the book vector
    book_features = feature_matrix.iloc[book_id:book_id+1]
    
    # Calculate similarity with all other books (in batches to save memory)
    batch_size = 1000
    n_batches = len(feature_matrix) // batch_size + 1
    similarities = []
    
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(feature_matrix))
        batch = feature_matrix.iloc[start_idx:end_idx]
        
        # Calculate similarity
        sim_batch = cosine_similarity(book_features, batch).flatten()
        similarities.extend(list(zip(range(start_idx, end_idx), sim_batch)))
    
    # Sort by similarity (excluding the book itself)
    similarities.sort(key=lambda x: x[1], reverse=True)
    similar_books = [pair[0] for pair in similarities if pair[0] != book_id][:n]
    
    # Return recommendations
    return df.iloc[similar_books][['title', 'author', 'rating']]

def collaborative_filtering(df, n=5):
    """Simple memory-efficient collaborative filtering"""
    # Create a smaller user-item matrix (simulate collaborative filtering)
    n_users = 100
    n_items = min(1000, len(df))
    
    # Sample some books
    sampled_books = df.iloc[:n_items].copy()
    
    # Generate random user preferences
    np.random.seed(42)
    user_preferences = np.random.rand(n_users, n_items).astype('float32')
    
    # Perform SVD on this small matrix
    svd = TruncatedSVD(n_components=10, random_state=42)
    user_factors = svd.fit_transform(user_preferences)
    item_factors = svd.components_.T
    
    # For a random user, get recommendations
    user_id = 0
    user_vector = user_factors[user_id]
    scores = np.dot(user_vector, item_factors.T)
    
    # Get top books
    top_items = np.argsort(scores)[::-1][:n]
    return sampled_books.iloc[top_items][['title', 'author', 'rating']]


=== Generating Recommendations ===

In [158]:
# ===== FINAL EVALUATION =====
print("\n=== Final Evaluation and Insights ===")
print("1. Best Classification Model:")
best_model_name = max([
    ("XGBoost", max(model_metrics.get("XGBoost", {}).items(), key=lambda x: x[1]["accuracy"])[1]["accuracy"] if "XGBoost" in model_metrics else 0),
    ("KNN", max(model_metrics.get("KNN", {}).items(), key=lambda x: x[1]["accuracy"])[1]["accuracy"] if "KNN" in model_metrics else 0),
    ("LogisticRegression", max(model_metrics.get("LogisticRegression", {}).items(), key=lambda x: x[1]["accuracy"])[1]["accuracy"] if "LogisticRegression" in model_metrics else 0),
], key=lambda x: x[1])[0]

print(f"The best classification model for this dataset is: {best_model_name}")
print("This model can be used for predicting whether a book will have a high rating (≥4.0).")

print("\n2. Recommendation Systems:")
print("- Content-based system: Best for recommending books similar to ones the user already likes")
print("- Collaborative filtering: Better for discovering new books based on user behavior patterns")
print("- Hybrid system: Combines strengths of both approaches for better recommendations")

print("\n3. Key Features Importance:")
if hasattr(xgb_model, 'feature_importances_'):
    feature_importance = xgb_model.feature_importances_
    feature_names = X_train.columns
    
    # Sort features by importance
    sorted_idx = np.argsort(feature_importance)[::-1]
    top_features = [(feature_names[i], feature_importance[i]) for i in sorted_idx[:10]]
    
    print("Top 10 important features for predicting book ratings:")
    for i, (feature, importance) in enumerate(top_features):
        print(f"{i+1}. {feature}: {importance:.4f}")

print("\n4. Model Training Efficiency:")
for model, time_taken in sorted(model_training_time.items(), key=lambda x: x[1]):
    print(f"- {model}: {time_taken:.2f} seconds")

print("\n=== Evaluation complete ===")
print("Recommendation system is ready for deployment.")

=== Final Evaluation and Insights ===

1. Best Classification Model:

The best classification model for this dataset is: XGBoost

This model can be used for predicting whether a book will have a high rating (≥4.0).

2. Recommendation Systems:

...

- KNN: 1.85 seconds

- KMeans: 4.50 seconds

- SVD: 17.69 seconds

=== Evaluation complete ===

Recommendation system is ready for deployment.

In [166]:

joblib.dump(xgb_model,           "xgb_model.pkl")
joblib.dump(kmeans_model,        "kmeans_model.pkl")
joblib.dump(knn_model,           "knn_model.pkl")
svd_transformer, logreg_on_svd = svd_models
joblib.dump(svd_transformer,     "svd_transformer.pkl")
joblib.dump(logreg_on_svd,       "svd_logistic_model.pkl")

print("✅ Saved: xgb_model.pkl, kmeans_model.pkl, knn_model.pkl, svd_transformer.pkl, svd_logistic_model.pkl")

✅ Saved: xgb_model.pkl, kmeans_model.pkl, knn_model.pkl, svd_transformer.pkl, svd_logistic_model.pkl

In [167]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

def prepare_numeric_features(df):
    df2 = df.copy()
    df2["log_pages"]        = np.log1p(df2["pages"])
    df2["log_reviews"]      = np.log1p(df2["reviews"])
    df2["log_totalratings"] = np.log1p(df2["totalratings"])
    df2["popularity_score"] = df2["rating"] * df2["log_totalratings"]
    df2["review_ratio"]     = df2["reviews"] / df2["totalratings"].replace(0, np.nan)
    return df2[[
        "log_pages","log_reviews","log_totalratings",
        "popularity_score","review_ratio"
    ]].fillna(0)

def recommend_books(df, genres, top_n=10):
    # — filter for candidates —
    df_clean = df.dropna(subset=["genre"]).reset_index(drop=True)
    df_clean["genre_list"] = df_clean["genre"].str.split(",").apply(lambda L: [g.strip() for g in L])
    exploded = df_clean.explode("genre_list")
    cands = exploded[exploded["genre_list"].isin(genres)].drop_duplicates("title")
    if cands.empty:
        print("⚠️  No books found for those genres.")
        return pd.DataFrame()
    idx = cands.index

    # — build feature‐matrix on the *entire* df, filling missing genre as "Other" —
    df_feats = df.copy().reset_index(drop=True)
    df_feats["genre"] = df_feats["genre"].fillna("Other")

    feats_num = prepare_numeric_features(df_feats)
    # label‐encode author & format
    le_a = LabelEncoder().fit(df_feats["author"])
    feats_num["author_encoded"] = le_a.transform(df_feats["author"])
    le_f = LabelEncoder().fit(df_feats["bookformat"].fillna("Unknown"))
    feats_num["format_encoded"] = le_f.transform(df_feats["bookformat"].fillna("Unknown"))

    # one‐hot the entire genre string (matches training)
    genre_dummies = pd.get_dummies(df_feats["genre"], prefix="genre")
    feats_full = pd.concat([feats_num, genre_dummies], axis=1)

    # — load your saved artifacts —
    xgb    = joblib.load("xgb_model.pkl")
    svd    = joblib.load("svd_transformer.pkl")
    logreg = joblib.load("svd_logistic_model.pkl")

    # — reindex to exactly XGB’s training columns (fill zeros for any missing) —
    feat_names = xgb.get_booster().feature_names
    feats_full = feats_full.reindex(columns=feat_names, fill_value=0)

    # — score with XGB —
    cands = cands.copy()
    cands["xgb_proba"] = xgb.predict_proba(feats_full.loc[idx])[:,1]

    # — project + score with SVD+LogReg —
    latent_all  = svd.transform(feats_full)
    latent_cand = latent_all[idx]
    cands["svd_proba"] = logreg.predict_proba(latent_cand)[:,1]

    # — optional semantic sim in SVD space —
    user_vec = latent_cand.mean(axis=0).reshape(1,-1)
    cands["sim_score"] = cosine_similarity(latent_cand, user_vec).flatten()

    # — blend & pick top-N —
    cands["final_score"] = 0.5*cands["xgb_proba"] + 0.5*cands["svd_proba"]
    top = cands.sort_values("final_score", ascending=False).head(top_n)

    print(f"✅ Found {len(top)} recommendations.")
    return top[[
        "title","author","genre","rating",
        "xgb_proba","svd_proba","sim_score","final_score"
    ]].reset_index(drop=True)



In [168]:
# ─ run it ─
recs = recommend_books(df, genres=["Romance", "Action"], top_n=10)
print(recs)

✅ Found 10 recommendations.

title  \
0                             E'S: Volume 5   
1                           Constantine Cay   
2                          Ripe for Revenge   
3                               Delta Pearl   
4                       The Genuine Article   
5                      No Place for a Woman   
6                               Winter Song   
7                      The Devil Of Talland   
8                In the House of Dark Music   
9  Employee Reward (People & organisations)   

                                            author  \
0  Satol Yuiga,Samantha Yamanaka,Satsuki Yamashita   
1                                 Catherine Dillon   
2                                      Isobel Carr   
3                                  Maureen Bronson   
4                                   Mary E. Butler   
5                                 Suzanne Michelle   
6                                     Lisa Plumley   
7                                Valentina Luellen   
8                                    Frances Lynch   
9                                Michael Armstrong   

                                            genre  rating  xgb_proba  \
0                     Action,Sequential Art,Manga    5.00   0.999898   
1                                         Romance    4.00   0.999864   
2  Romance,Historical Romance,Historical,Georgian    5.00   0.999804   
3                                         Romance    4.00   0.999527   
4                      Romance,Historical Romance    3.00   0.999524   
5                                         Romance    3.00   0.999522   
6                     Romance,Time Travel Romance    3.00   0.999432   
7                                         Romance    2.50   0.999357   
8                                  Romance,Gothic    3.33   0.998950   
9             Romance,Historical Romance,Business    4.67   0.998920   

   svd_proba  sim_score  final_score  
0        1.0   0.999999     0.999949  
1        1.0   0.999955     0.999932  
2        1.0   0.999996     0.999902  
3        1.0   0.999999     0.999763  
4        1.0   0.999999     0.999762  
5        1.0   0.999999     0.999761  
6        1.0   0.999999     0.999716  
7        1.0   0.999998     0.999679  
8        1.0   0.999996     0.999475  
9        1.0   0.999999     0.999460

In [169]:
def suggest_library_books(
    df,
    user_df,
    top_m_genres: int = 5,
    top_n_books: int = 5
):
    # 1) explode user_df into one row per (user, genre)
    uf = user_df.copy()
    def norm(g):
        if isinstance(g, str):
            return [x.strip() for x in g.split(",")]
        return list(g)
    uf["genre_list"] = uf["genres"].apply(norm)
    uf_exp = uf.explode("genre_list")

    # 2) count preferences
    genre_counts = uf_exp["genre_list"].value_counts()

    # 3) pick top M genres
    top_genres = genre_counts.head(top_m_genres).index.tolist()

    # 4) for each genre, get your recommend_books top-N
    suggestions = {}
    for genre in top_genres:
        # recommend_books(df, [genre], top_n_books) returns a DataFrame
        recs = recommend_books(df, genres=[genre], top_n=top_n_books)
        suggestions[genre] = recs

    return suggestions, genre_counts

# ─── Example Usage ────────────────────────────────────────────────────────────

# (Re‐use your 20‐user DataFrame from before)
user_data = pd.DataFrame({
    "user_id": [f"user_{i}" for i in range(1,21)],
    "genres": [
        "Fiction,Fantasy", "Romance,Nonfiction", "History", "Science Fiction,Mystery",
        "Fantasy,Young Adult", "Mystery,Thriller", "Business,Leadership",
        "Poetry,Art", "Science,Mathematics", "Biography,Memoir",
        "Self Help,Health", "Travel,Photography", "Cooking,Food",
        "Comics,Graphic Novels", "Religion,Spirituality", "Music",
        "Children’s", "Horror", "Classics,Philosophy", "Sports"
    ]
})

# Get suggestions
suggestions, genre_counts = suggest_library_books(
    df,
    user_data,
    top_m_genres=5,    # e.g. pick the 5 most‐popular genres among users
    top_n_books=5      # and get top‐5 books per genre
)

print("📊 User‐genre counts:\n", genre_counts.head(10), "\n")

for genre, rec_df in suggestions.items():
    print(f"── Top books to stock for genre: {genre} ──")
    print(rec_df[["title","author","rating","final_score"]], "\n")


✅ Found 5 recommendations.

✅ Found 5 recommendations.

✅ Found 5 recommendations.

✅ Found 5 recommendations.

✅ Found 5 recommendations.

...

title  \
0                               Les nouvelles sectes   
1                           Acts: An Inductive Study   
2             African Zion: Studies in Black Judaism   
3       Mulla Husayn: "Unfurling the Black Standard"   
4  Mysteries; Or, Glimpses Of The Supernatural Co...   

                       author  rating  final_score  
0               Alain Woodrow     4.0     0.999942  
1            Irving L. Jensen     4.0     0.999942  
2  Edith Bruder,Tudor Parfitt     5.0     0.999817  
3                  Ivan Lloyd     4.0     0.999809  
4      Charles Wyllys Elliott     4.0     0.999809

── Top books to stock for genre: Travel ──

title  \
0  Pictorial Guide To The Mountains Of Snowdonia ...   
1  Insiders' Guide to South Dakota's Black Hills ...   
2  America's Ancient Treasures: A Guide to Archeo...   
3                 Classic Walks In The Peak District   
4         The Companion Guide to the South of France   

                                              author  rating  final_score  
0                                       John Gillham     5.0     0.999950  
1  Thomas D. Griffith,Bert Gildart,Dustin D. Floy...     4.0     0.999943  
2                                    Franklin Folsom     4.0     0.999809  
3                                     Martin Collins     3.0     0.999774  
4                                    Archibald Lyall     3.5     0.999766

── Top books to stock for genre: Photography ──

title  \
0                           For the Love of Tasmania   
1                                      Egyptian Time   
2                   Medinas: Morocco's Hidden Cities   
3  American Politicians: Photographs from 1843 to...   
4    Portraits Of The Insane: The Case Of Dr Diamond   

                                author  rating  final_score  
0           Owen Hughes,Martin Gilmour    5.00     0.999947  
1        Robert Lyons,Najsib Ma=hfsu=z    3.00     0.999767  
2  Jean-Marc Tingaud,Tahar Ben Jelloun    4.00     0.998972  
3                       Susan Kismaric    3.25     0.998909  
4                     Adrienne Burrows    4.83     0.996859